In [1]:
# the copywright author is added, which is notebook specific
# for more information on codeowners and details, check CODEOWNERS
__copyright__ = "Copywright © 2025 Debmalya Pramanik (ZenithClown)"

<div align = "center">

# Basic Concepts & Usage of `NetworkX`

</div>

A *jupyter notebook* file exploring the optimization of share of business across plant-vendor combination with different constraints (MOQ, pack size, min. order, capacity) to meet the demand at a plant while minimizing the overall cost and time. Typically, this can be done using various optimization libaries as explained below.

In [2]:
import os   # miscellaneous os interfaces
import sys  # configuring python runtime environment
import json # json, i.e., dict based object manipulation
import math # basic mathematical operations, etc.

In [3]:
import random # generate random numbers for poc development

In [4]:
from typing import Iterable
from uuid import uuid4 as UUID4
from tqdm.auto import tqdm as TQ

### Module for Complex Network

**`NetworkX`** is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. In addition, we are using **`gravis`** which is great for visualization of graphs and has an inbuilt feature of conversion of a primitive `nx.Graph` to a [D3.js](https://d3js.org/) visualization for export/view inline in jupyter notebook.

In [5]:
import gravis as gv
import networkx as nx

### Module for Optimization

In [6]:
import pulp as p

## User Defined Function(s)

It is recommended that any UDFs are defined outside the scope of the *jupyter notebook* such that development/editing of function can be done more practically. As per *programming guidelines* as [`src`](https://fileinfo.com/extension/src) file/directory is beneficial in code development and/or production release. However, *jupyter notebook* requires *kernel restart* if any imported code file is changed in disc, for this frequently changing functions can be defined in this section.

**Getting Started** with **`PYTHONPATH`**

One must know what are [Environment Variable](https://medium.com/chingu/an-introduction-to-environment-variables-and-how-to-use-them-f602f66d15fa) and how to call/use them in your choice of programming language. Note that an environment variable is *case sensitive* in all operating systems (except windows, since DOS is not case sensitive). Generally, we can access environment variables from terminal/shell/command prompt as:

```shell
# macOS/*nix
echo $VARNAME

# windows
echo %VARNAME%
```

Once you've setup your system with [`PYTHONPATH`](https://bic-berkeley.github.io/psych-214-fall-2016/using_pythonpath.html) as per [*python documentation*](https://docs.python.org/3/using/cmdline.html#envvar-PYTHONPATH) is an important directory where any `import` statements looks for based on their order of importance. If a source code/module is not available check necessary environment variables and/or ask the administrator for the source files.

Most of the utility functions available in `PYTHONPATH` is tracked and maintained in [GIST.GitHub/ZenithClown](https://gist.github.com/ZenithClown) which provides detailed documentation and code snippets/example use cases etc. For more information, and category wise module check [github](https://github.com/ZenithClown/ZenithClown) repository.

In [7]:
sys.path.append(os.path.join("..")) # base of the repository

# methods/class objects to define graph components
from routicle.components import edges, nodes

# methods/class objects for core functionalities of scm & logistics applications
import routicle.core.networkx as rnx
import routicle.core.optimizer as ro

## Global Argument(s)

The global arguments are *notebook* specific, however they may also be extended to external libraries and functions on import. The *boilerplate* provides a basic ML directory structure which contains a directory for `data` and a separate directory for `output`. In addition, a separate directory (`data/processed`) is created to save processed dataset such that preprocessing can be avoided.

In [8]:
# define the number of nodes by type, for production
# the value should be fetched from a config file/dataframe
N_PLANTS, N_VENDORS = 5, 3

## Share of Business Optimization

In [9]:
plants = [
    nodes.ManufacturingUnits(name = "Vapi 78/79", rate = 0.0, demand = 12.0),
    nodes.ManufacturingUnits(name = "DAMAN UNIT 1", rate = 0.0, demand = 9.0),
    nodes.ManufacturingUnits(name = "SPAN INTERMEDIATES", rate = 0.0, demand = 4.0),
    nodes.ManufacturingUnits(name = "DAHEZ SEZ", rate = 0.0, demand = 1.0),
    nodes.ManufacturingUnits(name = "BADDI UNIT 1", rate = 0.0, demand = 4.0),
]

vendors = [
    nodes.SupplyPoints(name = "DPL", minorder = 5.0, moq = 5.0, maxcapacity = 100),
    nodes.SupplyPoints(name = "CJ Shah", minorder = 0.0, maxcapacity = 15.0),
    nodes.SupplyPoints(name = "PRASOL", minorder = 0.0, maxcapacity = 4.0),
]

dnodes = {node.name : node for node in plants + vendors}

In [10]:
# get the cost matrix from an underlying excel sheet/like wise
# the data shape must be of dimension of p x v where p = plants, v = vendors
# the iteration will be on for p in plants for v in vendors ... always for lookup
costmatrix = [
    # n-plants x n-vendors matrix, set cost and/or time for edges
    [87.24, 85.75, 85.50],
    [87.24, 85.75, 85.50],
    [87.24, 85.75, 85.50],
    [87.24, 85.00, 86.50],
    [91.49, 89.00, float("inf")]
]

timematrix = [[1.0] * N_VENDORS] * N_PLANTS

# dedges is a connection between the nodes to edges, and dynamically allocate cost and rate of production (if any)
dedges = {
    (v.name, p.name) : edges.TimeCostEdge(
        name = f"{v.name} --> {p.name}", unode = v, vnode = p,
        time = timematrix[pidx][vidx], cost = costmatrix[pidx][vidx],
        idgen = lambda un, vn : f"V2P_{un}_{vn}".upper(),
        useselfname = False, idgenargs = [v.cidx, p.cidx],
        indexposition = (pidx, vidx)
    )
    for pidx, p in enumerate(plants) for vidx, v in enumerate(vendors)
    if math.isfinite(costmatrix[pidx][vidx])
}

In [11]:
network = rnx.nxGraph(G = nx.DiGraph(), dnodes = dnodes, dedges = dedges) # created a directed graph, between vendor to plant to make time cost edge

In [12]:
fig = gv.d3(network.G, show_menu = True, show_node_image = True, node_label_size_factor = 0.50)
# fig.display(inline = True) # or view in browser

In [13]:
model = ro.PuLPModel(name = "SOBOptimizer", network = network)

In [14]:
demand, supply, supplyiter, demanditer = model.create_constraints(plants)

In [15]:
for item in supplyiter:
    model += p.lpSum(item["variables"]) >= item["object"].minorder, "S_{name}_MO".format(name = item["object"].cidx) # add min. order quantity

    maxcapacity = item["object"].maxcapacity
    maxcapacity = maxcapacity if maxcapacity != float("inf") else 1e3
    model += p.lpSum(item["variables"]) <= maxcapacity, "S_{name}_CAPACITY".format(name = item["object"].cidx)

In [16]:
for item in demanditer:
    model += p.lpSum(item["variables"]) == item["object"].demand, "P_{name}_DEMAND".format(name = item["object"].cidx)

In [17]:
print(f"Objective Function::\n\t{model.objective}", end = "\n\n")
print(f"No. of Variables = {len(model.nvariables):,} | Variable Names::\n\t{model.nvariables}", end = "\n\n")
print(f"No. of Constraints = {len(model.nconstraints):,} | Constraint Definition::\n{json.dumps(model.nconstraints, default = str, indent = 2)}")

Objective Function::
	89.0*V2P_CJSHAH_BADDIUNIT1 + 85.0*V2P_CJSHAH_DAHEZSEZ + 85.75*V2P_CJSHAH_DAMANUNIT1 + 85.75*V2P_CJSHAH_SPANINTERMEDIATES + 85.75*V2P_CJSHAH_VAPI7879 + 91.49*V2P_DPL_BADDIUNIT1 + 87.24*V2P_DPL_DAHEZSEZ + 87.24*V2P_DPL_DAMANUNIT1 + 87.24*V2P_DPL_SPANINTERMEDIATES + 87.24*V2P_DPL_VAPI7879 + 86.5*V2P_PRASOL_DAHEZSEZ + 85.5*V2P_PRASOL_DAMANUNIT1 + 85.5*V2P_PRASOL_SPANINTERMEDIATES + 85.5*V2P_PRASOL_VAPI7879

No. of Variables = 14 | Variable Names::
	[V2P_CJSHAH_BADDIUNIT1, V2P_CJSHAH_DAHEZSEZ, V2P_CJSHAH_DAMANUNIT1, V2P_CJSHAH_SPANINTERMEDIATES, V2P_CJSHAH_VAPI7879, V2P_DPL_BADDIUNIT1, V2P_DPL_DAHEZSEZ, V2P_DPL_DAMANUNIT1, V2P_DPL_SPANINTERMEDIATES, V2P_DPL_VAPI7879, V2P_PRASOL_DAHEZSEZ, V2P_PRASOL_DAMANUNIT1, V2P_PRASOL_SPANINTERMEDIATES, V2P_PRASOL_VAPI7879]

No. of Constraints = 11 | Constraint Definition::
{
  "S_CJSHAH_MO": "V2P_CJSHAH_BADDIUNIT1 + V2P_CJSHAH_DAHEZSEZ + V2P_CJSHAH_DAMANUNIT1 + V2P_CJSHAH_SPANINTERMEDIATES + V2P_CJSHAH_VAPI7879 >= -0.0",
  "S_CJSHA

In [ ]:
status = model.optimize()

# we've defined a positional index in an attribute, this
# can be used to return the matrix in the original order for representation
output = [[None] * N_VENDORS for _ in range(N_PLANTS)] # https://stackoverflow.com/a/2739564

for var in model.nvariables:
    edge = network.getbycidx(str(var), component = "edge")
    output[edge.indexposition[0]][edge.indexposition[1]] = p.value(var)

print(o)

In [21]:
# model.__reset_constraints__()